In [7]:
import os
os.chdir( "/home/lucasfuzato/CODING/WINE" )

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from itertools import product

from src.aux_fun import *

tests_df = pd.DataFrame( columns = [ "tree_num" , "num_features" ] )

# tree_size = [ 5 , 10 , 50 , 100 ]
# feature_select = [ "log2" , "sqrt"]

tree_size = [ 10 , 50 , 100 ]
feature_select = [ 3 , 6 , 9 ]
for i , ( tree_num , num_features ) in enumerate( product( tree_size , feature_select ) ):
    tests_df.loc[ i ] = pd.Series( [ tree_num , num_features ] , [ "tree_num" , "num_features" ] )
print( tests_df ) 

   tree_num  num_features
0        10             3
1        10             6
2        10             9
3        50             3
4        50             6
5        50             9
6       100             3
7       100             6
8       100             9


In [8]:
from sklearn.ensemble import RandomForestClassifier as rd_frst
seed_val = 47

depth , algo = 30 , "entropy"

X , y , X_2d , columns = class_get_data()

scores = [ "f1_score" , "precision" , "recall" ]
metrics = [ "mean" , "std" ]
data = { tup:list() for tup in product( scores , metrics ) }

for simu_id , ( tree_num , num_features ) in enumerate( product( tree_size , feature_select ) ):

    lst = []
    fold_iter = get_prepared_data( X , y , seed = seed_val , smote = False )
    for ( X_train , y_train ) , ( X_test , y_test ) in fold_iter:

        forest = rd_frst( n_estimators = tree_num , max_features = num_features, criterion = algo , max_depth = depth  )
        forest.fit( X_train , y_train )

        ts_cls = forest.predict( X_test )
        ts_f1 , prec , recall = bin_f1_score( ts_cls , y_test )

        lst.append( ( ts_f1 , prec , recall ) )
    
    arr = np.array( lst )
    means = arr.mean( axis = 0 )
    data[ ( "f1_score", "mean" ) ].append( means[ 0 ] )
    data[ ( "precision", "mean" ) ].append( means[ 1 ] )
    data[ ( "recall", "mean" ) ].append( means[ 2 ] )

    stds  = arr.std( axis = 0 )
    data[ ( "f1_score", "std" ) ].append( stds[ 0 ] )
    data[ ( "precision", "std" ) ].append( stds[ 1 ] )
    data[ ( "recall", "std" ) ].append( stds[ 2 ] )

final_data = pd.DataFrame( data )
print( final_data )

final_data.to_csv( "rand_forest.csv" , sep = ";" )

   f1_score           precision              recall          
       mean       std      mean       std      mean       std
0  0.719669  0.025619  0.798672  0.028112  0.556810  0.036917
1  0.720574  0.028614  0.795079  0.037355  0.563829  0.029106
2  0.708816  0.026539  0.785087  0.032844  0.549754  0.026941
3  0.749962  0.028034  0.823457  0.038723  0.592797  0.023931
4  0.740041  0.026146  0.804864  0.034982  0.598314  0.036120
5  0.744154  0.021392  0.806383  0.033658  0.606145  0.025893
6  0.751872  0.033972  0.823977  0.044238  0.597515  0.035056
7  0.756622  0.034655  0.826527  0.043960  0.605376  0.036339
8  0.747991  0.021838  0.809400  0.029421  0.610827  0.029048

